# 1 - Preparar as Tools para o Llama Index

In [45]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client)

In [46]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

add_data Adiciona novos dados à tabela do tomógrafo usando uma consulta SQL INSERT.

Argumentos:
    query (str): query SQL INSERT seguindo o seguinte formato:
        INSERT INTO tomografo (nomePaciente, idade, resultados)
        VALUES ('João Silva', 45, 'Glaucoma')
    
Esquema:
    - nomePaciente: campo de texto (exigido)
    - idade: campo de inteiro (exigido)
    - resultados: campo de texto (exigido)
    Observação: O campo 'idExame' é gerado automaticamente.

Retorno:
    bool: True se os dados foram inseridos corretamente, False caso contrário

Exemplo:
    >>> query = '''
    ... INSERT INTO tomografo (nomePaciente, idade, resultados)
    ... VALUES ('Alice Guerra', 25, 'Retinopatia diabética')
    ... '''
    >>> add_data(query)
    True

read_data Lê dados da tabela tomografo usando uma query SQL SELECT.

Argumentos:
    query (str, optional): SQL SELECT query. O comportamento default é: "SELECT * FROM tomografo;".
        Exemplos:
        - "SELECT * FROM tomografo"
    

# 2 - Construir o Agente

In [47]:
import nest_asyncio
nest_asyncio.apply()

In [48]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="qwen3:4b", request_timeout=120)
Settings.llm = llm

In [49]:
SYSTEM_PROMPT = """
Você é um assistente para chamada de ferramentas.

Antes de ajudar um usuário, você deve trabalhar com ferramentas 
para interagir com nossa base de dados.
"""

In [50]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agente HBr",
        description="Um agente que trabalha com a base de dados do tomógrafo.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )

    return agent

In [51]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [52]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(tools=mcp_tool)

# create the agent context
agent_context = Context(agent)

In [53]:
print("""Bem vindo ao servidor MCP de teste da HBr - CIIA!\n===================================================\n\nInsira abaixo seus comandos. Para sair, digite \"sair\".\n""")

while True:
    user_message = input("Digite a sua mensagem: ")

    if (user_message.lower() == "sair") :
        break

    print(f"\n Você: {user_message}")
    response = await handle_user_message(user_message, agent, agent_context, verbose=True)
    print(f"\nAgente CIIA: {response}")

Bem vindo ao servidor MCP de teste da HBr - CIIA!

Insira abaixo seus comandos. Para sair, digite "sair".


 Você: Leia todos os registros da tabela tomografo


ResponseError: memory layout cannot be allocated (status code: 500)